# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'local', 'met', 'travel_db', 'uk_food']


In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('66390f72cf5fa83562659879'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, '

In [10]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [11]:
# Create a dictionary for the new restaurant data
penang = {
   "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [12]:
# Insert the new restaurant into the collection
establishments.insert_one(penang)

InsertOneResult(ObjectId('6643a79db128cf3af3209e8e'), acknowledged=True)

In [13]:
# Check that the new restaurant was inserted
establishments.find_one({'Business Name' :'Penang Flavours'})

{'_id': ObjectId('663929d2b1460bbcd44e7bbc'),
 'Business Name': 'Penang Flavours',
 'BusinessTypeID': [1],
 'RatingValue': None}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [17]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID':1, 'BusinessType':1} 
results = establishments.find_one(query, fields)

pprint(results)


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66390f72cf5fa8356265987a')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [18]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({
    'Business Name' :'Penang Flavours'},
    {'$push':
         {'BusinessTypeID': 1}
    }
                         )
    

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# Confirm that the new restaurant was updated
query = {'Business Name' :'Penang Flavours'}

result = establishments.find_one(query)

pprint(result)

{'Business Name': 'Penang Flavours',
 'BusinessTypeID': [1, 1, 1],
 'RatingValue': None,
 '_id': ObjectId('663929d2b1460bbcd44e7bbc')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [32]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName' : 'Dover'}

print(f'Number of Establishments in Dover: {establishments.count_documents(query)}')

Number of Establishments in Dover: 994


In [36]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [37]:
# Check if any remaining documents include Dover
print(f'Number of Establishments in Dover: {establishments.count_documents(query)}')

Number of Establishments in Dover: 0


In [38]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('66390f72cf5fa83562659879'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [56]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({'geocode.latitude':{'$exists' : True}, 'geocode.longitude': {'$exists' : True}},
   [
        {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}},
        {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}
    ]
)

UpdateResult({'n': 38785, 'nModified': 38785, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [60]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [61]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, 
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [62]:
# Check that the coordinates and rating value are now numbers
establishments.find_one()

{'_id': ObjectId('66390f72cf5fa83562659879'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': 4,
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCo